In [ ]:
Generate text 

In [1]:
import numpy as np
import sys
import random
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


load text and convert to lowercase

In [2]:
filename = "masteren_engl.txt"
raw_text = open(filename, 'r', encoding = 'utf-8').read()
raw_text = raw_text.lower()

Create mapping of unique chars to integers, and a reverse mapping

In [3]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

We can summarize the dataset

In [4]:
n_chars = len(raw_text)
n_vocab = len(chars)

print("Total characters:", n_chars)
print("Total vocab:", n_vocab)

Total characters: 14671
Total vocab: 59


prepare the dataset of input to output pairs encoded, as integers


In [5]:

seq_length = 100
dataX = []
dataY = []

for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i : i +seq_length]
    seq_out = raw_text[i + seq_length]
    
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
    
    
    
n_patterns = len(dataX)
print("Total Patterns:", n_patterns)

Total Patterns: 14571


We can implement these steps as below

In [6]:
#reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
#normalize
X = X / float(n_vocab)
#one hot encode the output variable
y = np_utils.to_categorical(dataY)

Define the LSTM model

In [7]:
model = Sequential()
model.add(LSTM(256, input_shape = (X.shape[1], X.shape[2]), return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

load the network weights

In [8]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"

In [9]:
checkpoint = ModelCheckpoint(filepath, monitor = 'loss', verbose = 1, save_best_only = True, mode = 'min')

callbacks_list = [checkpoint]

Fit the model

In [10]:
model.fit(X, y, epochs = 10, batch_size = 64, callbacks = callbacks_list)

Epoch 1/10
14571/14571 [==============================] - 431s 30ms/step - loss: 3.0710

Epoch 00001: loss improved from inf to 3.07095, saving model to weights-improvement-01-3.0710-bigger.hdf5
Epoch 2/10
14571/14571 [==============================] - 428s 29ms/step - loss: 3.0088

Epoch 00002: loss improved from 3.07095 to 3.00881, saving model to weights-improvement-02-3.0088-bigger.hdf5
Epoch 3/10
14571/14571 [==============================] - 442s 30ms/step - loss: 2.9548

Epoch 00003: loss improved from 3.00881 to 2.95479, saving model to weights-improvement-03-2.9548-bigger.hdf5
Epoch 4/10
14571/14571 [==============================] - 539s 37ms/step - loss: 2.8330

Epoch 00004: loss improved from 2.95479 to 2.83304, saving model to weights-improvement-04-2.8330-bigger.hdf5
Epoch 5/10
14571/14571 [==============================] - 547s 38ms/step - loss: 2.7337

Epoch 00005: loss improved from 2.83304 to 2.73370, saving model to weights-improvement-05-2.7337-bigger.hdf5
Epoch 6/1

In [11]:
filename = "weights-improvement-10-2.4635-bigger.hdf5"

model.load_weights(filename)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

we will create a reverse mapping that we can use to convert the integers back to characters so that we can understand the prediction

In [12]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [21]:
# pick a random seed
import numpy as np
import random
import sys
start = np.random.randint(0, len(dataX) - 1)
pattern = dataX[start]
print("Seed:")

print( "\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# generate characters

for i in range(10):
    x = np.reshape(pattern, (1, len(pattern), 1))
    
    x = x/float(n_vocab)
    
    prediction = model.predict(x, verbose = 0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1 : len(pattern)]
    
    
print("\nDone")    

Seed:
" raising his arms for the
next blow.-- it's easy to say... those damned stones, there's too much of
t "
he sooe th
Done
